# Goal of notebook: save the timezones for each US region

In [ ]:
!pip install timezonefinder

In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import json
import geopandas as gpd
import matplotlib.pyplot as plt

import sys
sys.path.append("../../")
from load_data import get_region_geojson

us_regions = pd.read_csv("../../../data/us_region_meta_data.csv",index_col=0)

# get the geopandas dataframe of regions
region_geojson = get_region_geojson()
region_geojson_path = "./region_geojson.json"
with open(region_geojson_path, "w") as f:
    json.dump(region_geojson, f)
gdf = gpd.read_file(region_geojson_path)

In [4]:
def get_tzs_for_us_regions(gdf, us_regions):
    """
    gdf: geopandas df of all regions
    us_regions: dataframe containing just us_regions
    """
    from timezonefinder import TimezoneFinder
    
    # compute the region's centroid latitude and longitude:
    # we will get the timzeone of the centroid and assign
    # that to be the timezone of the region
    gdf["centroid"] = gdf["geometry"].centroid
    gdf["lng"] = gdf["centroid"].x
    gdf["lat"] = gdf["centroid"].y

    tzf = TimezoneFinder()
    us_regions_list = us_regions["region"].unique().tolist()

    region_tz_rows = []
    for idx, row in gdf.iterrows():
        region = row["region"]
        if region not in us_regions_list:
            continue
        region_tz_rows.append({
            "region": region,
            "tz": tzf.timezone_at(lng=row["lng"], lat=row["lat"])
        })
    return pd.DataFrame(region_tz_rows)

In [5]:
tzs_df = get_tzs_for_us_regions(gdf, us_regions)
tzs_df

/tmp/ipykernel_1198369/1409508894.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["centroid"] = gdf["geometry"].centroid


,region,tz
0,AECI,America/Chicago
1,AVA,America/Los_Angeles
2,AZPS,America/Phoenix
3,BANC,America/Los_Angeles
4,BPA,America/Los_Angeles
...,...,...
83,TPWR,America/Los_Angeles
84,TVA,America/Chicago
85,WACM,America/Denver
86,WALC,America/Phoenix


In [7]:
# store timezones dataframe
import os
os.chdir('/home/eda.elliott/git/watttime-python-client-aer-algo/')
from data.s3 import s3_utils
s3 = s3_utils()
s3.store_csvdataframe(tzs_df, "us_region_timezones.csv")

Successful S3 put_object response. Status - 200


----

In [47]:
# example of how to tzconvert
pd_utc_tz = pd.Timestamp.now(tz="UTC")
print(pd_utc_tz)
print(pd_utc_tz.tz_convert("America/Denver"))

2024-07-13 14:36:39.077155+00:00
2024-07-13 08:36:39.077155-06:00
